In [1]:
from scipy.io import loadmat
from scipy import sparse
from classify import *

import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
from dgl import DGLGraph

from functools import partial
import argparse
import time
import numpy as np

In [2]:
# load graph data
data = loadmat('data/ACM3025.mat')
num_nodes = data['PTP'].shape[0]
num_rels = 2
labels = torch.from_numpy(data['label']).long()
feats = torch.from_numpy(data['feature']).float()
num_classes = labels.shape[1]
labels = labels.nonzero()[:, 1]
train_idx = data['train_idx']
test_idx = data['test_idx']
val_idx = data['val_idx']

In [3]:
data['PAP'][0].sum()

18.0

In [4]:
data['PAP'] = sparse.csr_matrix(data['PAP'] - np.eye(num_nodes))
data['PLP'] = sparse.csr_matrix(data['PLP'] - np.eye(num_nodes))

In [5]:
np.asarray(data['PAP'].sum(axis=1).reshape(-1))[0]

array([17.,  7.,  7., ...,  6., 20.,  8.])

In [6]:
np.asarray(data['PLP'].sum(axis=1).reshape(-1))[0]

array([   4.,  308., 1198., ..., 1198.,  308., 1198.])

In [7]:
g = DGLGraph()
g.add_nodes(num_nodes)

In [8]:
data['PAP'][0, :100].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [9]:
data['PAP'].indices

array([   8,   20,   51, ..., 2948, 2983, 2991], dtype=int32)

In [10]:
(data['PAP'].toarray()[0,:]-data['PAP'][0, :].toarray()).sum()

0.0

In [11]:
np.argwhere(data['PAP'] == 1)

array([[   0,    8],
       [   0,   20],
       [   0,   51],
       ...,
       [3024, 2948],
       [3024, 2983],
       [3024, 2991]], dtype=int32)

In [12]:
np.argwhere(data['PLP'] == 1)

array([[   0,   75],
       [   0,  434],
       [   0,  534],
       ...,
       [3024, 3020],
       [3024, 3021],
       [3024, 3022]], dtype=int32)

In [13]:
edge_list_1 = np.argwhere(data['PAP'] == 1)
src_1, dst_1 = tuple(zip(*edge_list_1))

In [14]:
edge_list_1.shape[0]

26256

In [15]:
edge_list_2 = np.argwhere(data['PLP'] == 1)
edge_list_2.shape[0]

2207736

In [16]:
edge_list_2.shape[0] // 220 * 220

2207700

In [17]:
idx = list(range(0, 2207700, 220))
edge_list_2_2 = edge_list_2[idx, :]

In [18]:
edge_list_2_2.shape[0]

10035